In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import re
from collections import Counter
from nltk.stem import porter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from bs4 import BeautifulSoup

In [3]:
import scipy as sp
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  TfidfVectorizer, CountVectorizer

In [4]:
import gensim
from gensim.models.doc2vec import Doc2Vec
from gensim.test.utils import get_tmpfile

In [5]:
data = pd.read_json('data/train.json')

In [6]:
data = data.reset_index()
data.rename(columns={'index':'doc_id'}, inplace=True) 

# Features

### CountVectorizer 

In [7]:
def get_all_features(data):
    all_features = []
    for feature_array in data['features'].values:
        if len(feature_array) > 0:
            for features in feature_array:
                all_features.append(features)
    return all_features

In [8]:
all_features = get_all_features(data)

In [9]:
data['num_of_features'] =data['features'].apply(len)

In [10]:
cv = CountVectorizer(stop_words='english', min_df=18, max_df = 0.9)
cv_fit = cv.fit_transform(all_features)
feature_name = cv.get_feature_names()

### TfidfVectorizer

In [11]:
import re
def word_to_phrase(flist):
    plist = []
    if len(flist) > 0:
        for feature in flist:
            feature = re.sub('[_]',' ',feature)
            feature = feature.strip()
            feature_p = ''.join(feature.split(' '))
            plist.append(feature_p)
    return plist

In [12]:
def list_to_string(flist):
    string = ' '.join(flist)
    return string

In [13]:
data['features_phr'] = data['features'].apply(word_to_phrase)
data['features_phr_str'] = data['features_phr'].apply(list_to_string)

In [14]:
phr_tfidf = TfidfVectorizer(min_df=2, max_features=200, strip_accents='unicode', lowercase=True, token_pattern=r'\w{3,}', stop_words='english')  
phr_tfidf_fit =phr_tfidf.fit_transform(data['features_phr_str'])
phr_names = [x for x in phr_tfidf.get_feature_names()]
data['features_phr_vec'] = pd.Series(phr_tfidf_fit.toarray().tolist())

In [15]:
data['features_str'] = data['features'].apply(list_to_string)

In [16]:
wd_tfidf = TfidfVectorizer(min_df=2, max_features=300, strip_accents='unicode', lowercase=True, token_pattern=r'\w{3,}', stop_words='english')  
wd_tfidf_fit = wd_tfidf.fit_transform(data['features_str'])
wd_names = [x for x in wd_tfidf.get_feature_names()]
data['features_vec'] = pd.Series(wd_tfidf_fit.toarray().tolist())

In [17]:
def extract_features(df):
    classifier = {
        "dogs": ("dogs", "dog"),
        "cats": ("cats", "cat"),
        "allowed":("allowed","allow"),
        "nofee": ("no fee", "no-fee", "no  fee", "nofee", "no_fee"),
        "lowfee": ("reduced_fee", "low_fee", "reduced fee", "low fee"),
        "security": ("concierge", "doorman", "housekeep", "in_super"),
        "laundry": ("laundry", "lndry"),
        "health": ("health", "gym", "fitness", "training","wheelchair"),
        "transport": ("train", "subway", "transport"),
        "parking": ("parking"),
        "hardwood": ("hardwood"),
        "utilities": ("utilities", "heat water", "water included","garden",'Dishwasher',)
    }

    def search_feature(feature):
        return lambda s: int(any([x in s for x in feature]))

    features = df["features"].apply(list_to_string)   # convert features to string
    features = features.str.lower()
    for key in classifier:
        df['feature_'+key] = features.apply(search_feature(classifier[key]))

    return df

In [18]:
data = extract_features(data)

### Wordcloud

In [19]:
def word_cloud(all_features):
    string = ' '.join(all_features).strip()
    wordcloud = WordCloud(max_words=50,collocations = False).generate(string)
    return plt.imshow(wordcloud)

word_cloud(all_features)
plt.savefig('feature_wordcloud.png')

# Description

In [20]:
def clean_text(text):
    # acronym
    text = re.sub(r"br\s", "bedroom", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"cannot", "can not ", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ve ", " have ", text)
    text = re.sub(r"n\'t", " not ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    # symbol
    text = re.sub(r"&", " and ", text)
    text = re.sub(r"\|", " or ", text)
    text = re.sub(r"=", " equal ", text)
    text = re.sub(r"\+", " plus ", text)
    text = re.sub(r"\$", " dollar ", text)
    # others
    text = re.sub(r"[^a-zA-Z]", " ",text)
    # extra \s
    text = " ".join(text.split())

    return text

In [21]:
def remove_stops(text):
    return [w for w in text.split(' ') if w not in stopwords.words('english')]

In [22]:
data['description_words'] = data['description'].apply(clean_text)

In [23]:
# data['description_words'] = data['description_words'].apply(remove_stops)

### tfidf 

In [24]:
desc_tfidf = TfidfVectorizer(min_df=20, max_features=50, strip_accents='unicode',lowercase =True,
                        analyzer='word', token_pattern=r'\w{5,}', ngram_range=(1, 3),  sublinear_tf=True, stop_words = 'english')  
desc_tfidf_fit =desc_tfidf.fit_transform(data['description'])
desc_name = [x for x in desc_tfidf.get_feature_names()]

In [25]:
data['description_vector'] = pd.Series(desc_tfidf_fit.toarray().tolist())

### doc2vec 

In [26]:
TaggededDocument = gensim.models.doc2vec.TaggedDocument

In [27]:
def addTag(descr):
    train=[]
    for i, doc in enumerate(descr):
        word_list = doc.split(' ')
        document = TaggededDocument(word_list, tags=[i])
        train.append(document)
    return train

In [28]:
taggedDoc = addTag(list(data['description_words']))

In [29]:
doc2vec = Doc2Vec(taggedDoc, vector_size=120, min_count = 20, window = 4, negative=5, sample = 1e-5, workers=4)

In [30]:
doc2vec.train(taggedDoc, total_examples=doc2vec.corpus_count, epochs=10)

In [31]:
fname = get_tmpfile("doc2vec_model")
doc2vec.save(fname)

In [32]:
doc2vec[2103]

array([-0.01114471,  0.03137974, -0.0068664 , -0.02417429,  0.03021809,
       -0.00219568,  0.0410921 , -0.01670329, -0.0061246 ,  0.01358639,
        0.01669458,  0.06767851,  0.03689447, -0.02858024,  0.00603091,
        0.02129531, -0.05357992, -0.03509167,  0.03033374,  0.00853469,
       -0.01573276,  0.00536507, -0.00101561, -0.01317887, -0.05653643,
        0.0159785 , -0.01006296,  0.03836722, -0.04459468, -0.0118534 ,
       -0.0067483 ,  0.03532721,  0.00753242, -0.02504994,  0.00463452,
       -0.00285272, -0.01051767,  0.02238576, -0.01776482,  0.00550217,
        0.0101871 , -0.00919038,  0.02677433,  0.01168147,  0.02775351,
        0.01532259, -0.00510034, -0.02447276,  0.06987439,  0.00216225,
        0.02524135,  0.01665253,  0.02794077, -0.05600229, -0.02085562,
        0.00520814,  0.00495165, -0.01517904,  0.02602663,  0.01796814,
        0.01056923,  0.00351833,  0.00881648,  0.00930572,  0.02837259,
       -0.03918439, -0.01132496,  0.0179447 , -0.00335504, -0.01

### artificial extraction

In [33]:
def extract_description(df):
    classifier = {
        "positive":("beautiful","great",'luxury'),
        "apartment": ("apartment","building"),
        "security": ("access", "doorman", "in_super"),
        "elevator": ("elevator"),
        "stainless": ("stainless"),
        "steel'": ("steel"),
        "dishwasher": ("dishwasher"),
        "good floor": ("granite","marble"),
        "renovated": ("renovated")
    }

    def search_desc(desc):
        return lambda s: int(any([x in s for x in desc]))

    desc = df["description"]
    for key in classifier:
        df['desc_'+key] = desc.apply(search_desc(classifier[key]))

    return df

In [34]:
data = extract_description(data)

In [35]:
data

doc_id  bathrooms  bedrooms                       building_id  \
0          10        1.5         3  53a5b119ba8f7b61d4e010512e0dfc85   
1       10000        1.0         2  c5c8a357cba207596b04d1afd1e4f130   
2      100004        1.0         1  c3ba40552e2120b0acfc3cb5730bb2aa   
3      100007        1.0         1  28d9ad350afeaab8027513a3e52ac8d5   
4      100013        1.0         4                                 0   
5      100014        2.0         4  38a913e46c94a7f46ddf19b756a9640c   
6      100016        1.0         2  3ba49a93260ca5df92fde024cb4ca61f   
7      100020        2.0         1  0372927bcb6a0949613ef5bf893bbac7   
8      100026        1.0         1  a7efbeb58190aa267b4a9121cd0c88c0   
9      100027        2.0         4                                 0   
10     100030        1.0         0                                 0   
11      10004        1.0         1                                 0   
12     100044        1.0         2  67c9b420da4a365bc26a6cd0ef4a5320   
13     100048        2.0         2                                 0   
14      10005        1.0         1                                 0   
15     100051        1.0         0  bfb9405149bfff42a92980b594c28234   
16     100052        1.0         2  642cc2c920512ffe2a74c28122f8b47f   
17     100053        1.0         1                                 0   
18     100055        1.0         4  cc4c6ae9225df6d2395c4e16c235f7ab   
19     100058        1.0         1  dc3cae15729b48fec3394f9295671991   
20     100062        1.0         3  205f95d4a78f1f3befda48b89edc9669   
21     100063        1.0         2  ecea86b79afa539505aa4bad3ff449c6   
22     100065        1.0         1  1a6cf9b71da65cdc0cfd5015a75317ac   
23     100066        1.0         1  a6200d7448037cfee809b2451219f879   
24      10007        2.0         4  05b871a1e1e0368391160bcefc608e00   
25     100071        3.5         4  45b4143a9841eb63d7203c4f21a64ff7   
26     100075        1.0         1  d68497ff1f5ac77f143e5f21817ed6fb   
27     100076        1.0         1  ec447453d3b8033e14a7b54ba1e13e02   
28     100079        1.0         1  f06ad2f6f1a821c8efc03dc195d369df   
29     100081        2.0         2  f115a9bff3a9c4e6eedc9114374b3d74   
...       ...        ...       ...                               ...   
49322   99915        1.0         1                                 0   
49323   99917        1.0         1  e243886a4cac69881c07c6a1f38185bf   
49324   99919        1.0         0  da2f861e30b1391e8368438efddaa2a7   
49325   99921        1.0         2  fc0676a4d4fdb4e767faf6414975a0cb   
49326   99923        2.0         2                                 0   
49327   99924        1.0         1  93cd24891f8423d45ac587ab1fdb1225   
49328   99931        1.0         1                                 0   
49329   99933        1.0         2  80a120d6bc3aba97f40fee8c2204524b   
49330   99935        1.0         1                                 0   
49331   99937        1.5         0  d48767c37a934daaf0bbb0e58c755d0c   
49332    9994        1.0         2                                 0   
49333   99953        1.0         1  e243886a4cac69881c07c6a1f38185bf   
49334   99956        2.0         2                                 0   
49335   99960        1.0         1  6f1e2020bd0e4ff48d3a5d482944a753   
49336   99961        2.0         3  ae3e5e3d93ea18140752078f9a3177af   
49337   99964        1.0         0  f8009151babfad197b48f6922eec99f2   
49338   99965        2.0         3  a485c2deb62ad3c9c76edebd532a446c   
49339   99966        1.0         3  a36895b9975c6991ca0fa1d9a99023a1   
49340   99979        1.0         1  e4490b8e6a410d39d38e40e572243a64   
49341   99980        1.0         0  d16f875956b264d5920d8424631091d4   
49342   99982        1.0         1  6134e7c4dd1a98d9aee36623c9872b49   
49343   99984        1.0         1  8e7bd3cbdd67a0dcc719224a41699d2b   
49344   99986        1.0         1  399f92c2a78ada2f01c8414ce3bc909b   
49345   99987        1.0         1             